EL OUAKYL EL MEHDI IID3

# Exercice 1

**Faits** :
- Un oiseau est un animal qui peut voler.
- Un mammifère est un animal qui allaite ses petits.
- Une chauve-souris est à la fois un mammifère et un oiseau.
- Les chauves-souris ne pondent pas d’œufs.

**Objectif** : construire un réseau sémantique, montrer propriétés héritées et conflits.

In [1]:
isa = {
    'oiseau': ['animal'],
    'mammifere': ['animal'],
    'chauve_souris': ['oiseau', 'mammifere']
}
properties = {
    'oiseau': ['peut_voler'],
    'mammifere': ['allaite_ses_petits'],
    'chauve_souris': ['ne_pond_pas_d_oeufs']
}

def inherited_properties(node, isa, properties):
    seen = set()
    stack = [node]
    inherited = []
    while stack:
        n = stack.pop()
        if n in properties:
            for p in properties[n]:
                if p not in inherited:
                    inherited.append(p)
        for parent, parents in isa.items():
            pass
    def traverse(node):
        for parent in isa.get(node, []):
            if parent not in seen:
                seen.add(parent)
                for p in properties.get(parent, []):
                    inherited.append(p)
                traverse(parent)
    traverse(node)
    return inherited

print('Proprietes heritees par la chauve-souris :', inherited_properties('chauve_souris', isa, properties))
print('Proprietes propres :', properties.get('chauve_souris'))

Proprietes heritees par la chauve-souris : ['ne_pond_pas_d_oeufs', 'peut_voler', 'allaite_ses_petits']
Proprietes propres : ['ne_pond_pas_d_oeufs']


# Exercice 2

Règles :
- Tous les poissons vivent dans l’eau.
- Tous les animaux qui vivent dans l’eau peuvent nager.
- Les dauphins sont des mammifères.
- Les mammifères sont des animaux.
- Les dauphins vivent dans l’eau.

Démontrer (par propagation d'héritage) que les dauphins peuvent nager.

In [2]:

rules = [
    ('poisson', 'vit_dans', 'eau'),
    ('animal_vit_dans_eau', '=>', 'peut_nager'),
    ('dauphin', 'isa', 'mammifere'),
    ('mammifere', 'isa', 'animal'),
    ('dauphin', 'vit_dans', 'eau')
]

facts = set([
    ('dauphin', 'isa', 'mammifere'),
    ('mammifere', 'isa', 'animal'),
    ('dauphin', 'vit_dans', 'eau'),
    ('poisson', 'vit_dans', 'eau') 
])

inferred = True
while inferred:
    inferred = False
    for (s,p,o) in list(facts):
        if p == 'vit_dans' and o == 'eau':
            if (s, 'peut', 'nager') not in facts:
                facts.add((s, 'peut', 'nager'))
                inferred = True

print('Faits finaux (extraits pertinents):')
for f in sorted(facts):
    if f[1] in ('isa','vit_dans','peut'):
        print(f)

print('\nConclusion: dauphin peut nager ? ->', ('dauphin','peut','nager') in facts)

Faits finaux (extraits pertinents):
('dauphin', 'isa', 'mammifere')
('dauphin', 'peut', 'nager')
('dauphin', 'vit_dans', 'eau')
('mammifere', 'isa', 'animal')
('poisson', 'peut', 'nager')
('poisson', 'vit_dans', 'eau')

Conclusion: dauphin peut nager ? -> True


# Exercice 3

Représenter :
- Une personne possède un âge (valeur numérique).
- Un étudiant est une personne qui étudie un domaine.
- Un enseignant est une personne qui enseigne un domaine.
- Un professeur est à la fois enseignant et chercheur.
- Le domaine enseigné doit être le même que le domaine étudié pour un même cours.

Construisez le réseau et identifiez le type de contrainte.

In [3]:

classes = {
    'Personne': {'attributes': ['age']},
    'Etudiant': {'parents': ['Personne'], 'attributes': ['etudie']},
    'Enseignant': {'parents': ['Personne'], 'attributes': ['enseigne']},
    'Chercheur': {'parents': ['Personne']},
    'Professeur': {'parents': ['Enseignant','Chercheur']}
}
constraint = "Pour un meme cours C: domaine(enseigne, C) == domaine(etudie, C)  (contrainte d'egalite)"

print('Classes et heritage:')
for c,v in classes.items():
    print('-', c, '-> parents:', v.get('parents', []), 'attributes:', v.get('attributes', []))
print('\nType de contrainte:', constraint)

Classes et heritage:
- Personne -> parents: [] attributes: ['age']
- Etudiant -> parents: ['Personne'] attributes: ['etudie']
- Enseignant -> parents: ['Personne'] attributes: ['enseigne']
- Chercheur -> parents: ['Personne'] attributes: []
- Professeur -> parents: ['Enseignant', 'Chercheur'] attributes: []

Type de contrainte: Pour un meme cours C: domaine(enseigne, C) == domaine(etudie, C)  (contrainte d'egalite)


# Exercice 4

Réseau donné et propagation d'activation :
- Chien → est un → Animal
- Chat → est un → Animal
- Animal → est un → ÊtreVivant
- Chien → lié À → Os
- Chat → lié À → Lait
- Os → type → Nourriture

Activation initiale: Chien (1.0). Niveau = 3, facteur = 0.5

In [4]:

from collections import defaultdict, deque

edges = {
    'Chien': [('isa','Animal'), ('lie','Os')],
    'Chat': [('isa','Animal'), ('lie','Lait')],
    'Animal': [('isa','EtreVivant',)],
    'Os': [('type','Nourriture')]
}

def propagate(initial, max_level=3, factor=0.5):
    activation = defaultdict(float)
    activation[initial] = 1.0
    queue = deque([(initial, 0)])
    while queue:
        node, level = queue.popleft()
        if level >= max_level: 
            continue
        for rel in edges.get(node, []):
            target = rel[1]
            add = activation[node] * factor
            if add > activation[target]:
                activation[target] = round(add, 4)
            queue.append((target, level+1))
    return dict(activation)

act = propagate('Chien', max_level=3, factor=0.5)
act['Chien'] = 1.0
print('Activations finales (arrondies) :')
for k,v in sorted(act.items(), key=lambda x: x[0]):
    print(f"{k}: {v}")

Activations finales (arrondies) :
Animal: 0.5
Chien: 1.0
EtreVivant: 0.25
Nourriture: 0.25
Os: 0.5


# Exercice 5

Faits :
- Les oiseaux peuvent voler.
- Les oiseaux nocturnes chassent la nuit.
- Les hiboux sont des oiseaux nocturnes.
- Les pingouins sont des oiseaux mais ne volent pas.
- Les hiboux ne vivent pas dans l’eau.

Déterminer pour Pingouin et Hibou les propriétés héritées finales.

In [5]:

isa = {
    'oiseau': ['animal'],
    'oiseau_nocturne': ['oiseau'],
    'hibou': ['oiseau_nocturne'],
    'pingouin': ['oiseau']
}
properties = {
    'oiseau': ['peut_voler'],
    'oiseau_nocturne': ['chasse_la_nuit'],
    'hibou': ['ne_vit_pas_dans_eau'],
    'pingouin': ['ne_peut_pas_voler']
}

def resolve(node):
    inherited = []
    visited = set()
    def traverse(n):
        for p in isa.get(n, []):
            if p not in visited:
                visited.add(p)
                for prop in properties.get(p, []):
                    if prop not in inherited:
                        inherited.append(prop)
                traverse(p)
    specific = properties.get(node, [])
    traverse(node)
    if 'ne_peut_pas_voler' in specific and 'peut_voler' in inherited:
        inherited.remove('peut_voler')
    # combine
    final = specific + [p for p in inherited if p not in specific]
    return final

print('Pingouin:', resolve('pingouin'))
print('Hibou:', resolve('hibou'))

Pingouin: ['ne_peut_pas_voler']
Hibou: ['ne_vit_pas_dans_eau', 'chasse_la_nuit', 'peut_voler']


# Exercice 6

Traduire en réseau sémantique :
- (∀x) Étudiant(x) → Personne(x)
- (∀x) Étudiant(x) → ¬Travailleur(x)
- Étudiant(Ali)

In [6]:

predicates = {
    'Etudiant': set(['Ali']),
    'Personne': set(),
    'Travailleur': set()
}

for x in list(predicates['Etudiant']):
    predicates['Personne'].add(x)
    if x in predicates['Travailleur']:
        predicates['Travailleur'].remove(x)

print('Personne:', predicates['Personne'])
print('Travailleur:', predicates['Travailleur'])

Personne: {'Ali'}
Travailleur: set()


# Exercice 7

Réseau médical simplifié :
- Fièvre → symptôme → Maladie
- Toux → symptôme → MaladieRespiratoire
- Grippe → isa → MaladieRespiratoire
- Grippe → hasSymptom → Fièvre, Toux
- Pneumonie → isa → MaladieRespiratoire
- Pneumonie → hasSymptom → Fièvre, DouleurPoitrine

Si un patient présente fièvre + toux, quelles maladies sont possibles ?

In [7]:

diseases = {
    'Grippe': {'Fievre','Toux'},
    'Pneumonie': {'Fievre','DouleurPoitrine'},
}

patient_symptoms = {'Fievre','Toux'}

possible = []
for d,s in diseases.items():
    if patient_symptoms.issubset(s):
        possible.append(d)

print('Patient symptoms:', patient_symptoms)
print('Maladies possibles (requierent au moins ces symptomes):', possible)

Patient symptoms: {'Fievre', 'Toux'}
Maladies possibles (requierent au moins ces symptomes): ['Grippe']
